# Proyecto Final 

### Asignatura: 
        Portafolios de Inversión 

### Integrantes: 
        1) Iván Andrés Arellano (714383)
        2) Fernando Aguilar Carvajal (IF724031)
        3) Ana Esmeralda Rodríguez Rodríguez (709288)


In [86]:
ETFS = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX','VOX.MX']

Stoks = ['BBVA.MX','AC.MX','AMXL.MX',
         'BIMBOA.MX','GAPB.MX','C.MX',
         'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX']

tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
           'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
           'BIMBOA.MX','GAPB.MX','C.MX',
           'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX' ]

## Explicacíon elección de activos

En nuestro portafolio elegimos utilizar ETF´s y acciones todos dentro de la bolsa mexicana de valores para asi formar un portafolio con posibilidades de acceso para todos los mexicanos en los distintos brokers y casas de bolsa disponibles en el pais, el portafolio busca estar diversificado abarcando varios sectores como el bancario, telecomunicaciones, productos de consumo, salud, tecnologia, etc.

### - ETF´s:
>
> - **XLK.MX ETF**´s basado en inversión en empresas de tecnologia.
> - **VOO.MX ETF´S** que replica al S&P 500 con lo que tenemos cubierto buena parte del mercado americano.
> - **VGT.MX ETF´s** que busca replicar al indice MSCI US Investable Market Information Technology 25/50 Index, basado en empresas de tecnologia de información.
> - **VHT.MX ETF´s** que busca replicar al indice MSCI US Investable Market Health Care 25/50 Index, basado en empresas del sector salud.
> - **VOX.MX ETF´s** que busca replicar al indice MSCI US Investable Market Communication Services 25/50 Index, basado en empresas del sector de comunicaciones.
### - Empresas
> - **BBVA.MX** empresa del sector bancario, una de las más grandes de México.
> - **AC.MX** empresa del sector de productos de consumo, dedicada a la comercializacion principalmente de bebidas no alcoholicas, snacks y otros alimentos, siendo una de las mas grandes de México y todo America Latina.
> - **AMXL.MX** empresa del sector de telecomunicaciones, una de las más grandes de de America Latina y la empresa con mayor peso dentro de principal indice de mercado de México (IPC).
> - **BIMBOA.MX** empresa del sector alimenticio con presencia en más de 30 paises en America, Europa, Asia y Africa.
> - **GAPB.MX** empresa del sector de transportes, con administración y operacion de mas de 10 Aeropuertos en México y el caribe.
> - **C.MX** empresa del sector bancario con presencia en todo el mundo.
> - **AEROMEX.MX** empresa del sector aereo, con una flota de mas de 80 aeronaves, siendo una de las empresas lideres en el sector dentro de México.
> - **WALMEX.MX** empresa de tiendas de ventas minoristas con más de 3 mil tiendas en México y Centroamerica.
> - **SAN.MX** empresa del sector bancario.
> - **GMEXICOB.MX** conglomerado que abarca desde el sector minero, transporte como ferrocariles e infrastructuras petroleras, de energia, etc.

In [87]:
# Se importan librerías 
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.optimize as opt
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader.data as web

In [88]:
# Funciones: 

# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

# Función Varianza
def varianza(w,Sigma):
    return w.T.dot(Sigma).dot(w) 

# Función objetivo
def menos_RS(w,Eind,Sigma,rf):
    Ep = Eind.T.dot(w)
    sp = (w.T.dot(Sigma).dot(w))**.5
    RS = (Ep-rf)/sp
    return -RS

# Función Cambio Porcentual
def pct_chng(VA: 'Valor Actual',
             VI: 'Valor Inicial'):
    
    ret = np.round(((VA-VI)/VI), 2)
    return ret


def simulacion_Portafolio(ret: 'Retornos',
                       rf: 'Tasa Libre de Riesgo',
                       g: 'Grado de Aversión al Riesgo'):
    
    global pesos, summary, w_EMV, w_opt, E_EMV, s_EMV,tickers
        
    #Rendimientos y volatilidades esperadas
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                 'Vol':((252)**(.5))*ret.std()})
        
    cov_ret = ret.cov() # Matriz de varianza - covarianza
    
    corr = ret.corr() # Matriz de correlación
    
    S = np.diag(summary['Vol'].values)
    Sigma = S.dot(corr).dot(S)
    Eind = summary['Media'].values
    
    n = len(Eind) # Número de activos    
    w0 = np.ones((n,))/n # Dato inicial   
    bnds = ((0, 1),)*n # Cotas de las variables    
    cons={'type':'eq','fun':lambda w:w.sum()-1} # Restricciones
    
    # Portafolio de mínima varianza
    minvar = minimize(fun=varianza,
                      x0=w0,
                      args=(Sigma,),
                      bounds=bnds,
                      constraints=cons,
                      tol=1e-5)
    
    w_minvar = minvar.x
    
    E_minvar = Eind.T.dot(w_minvar)
    s_minvar = (w_minvar.T.dot(Sigma).dot(w_minvar))**.5
    RS_minvar = (E_minvar - rf)/s_minvar

    # Portafolio EMV
    EMV = minimize(fun = menos_RS,
                   x0 = w0,
                   args = (Eind, Sigma, rf),
                   bounds = bnds,
                   constraints = cons,
                   tol = 1e-5)
    w_EMV = EMV.x
    
    # Pesos, rendimiento y riesgo del portafolio EMV
    E_EMV = Eind.T.dot(w_EMV)
    s_EMV = (w_EMV.T.dot(Sigma).dot(w_EMV))**0.5
    RS_EMV = (E_EMV - rf)/s_EMV
    
    #Asignación de capital
    w_opt = (E_EMV - rf)/(g*s_EMV**2)
    
    pesos = [w_EMV[i]*w_opt for i in range(len(w_EMV))]
    pesos.append(1-w_opt)
    pesos = np.round(pesos, 4)
    
    tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
              'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
              'BIMBOA.MX','GAPB.MX','C.MX',
              'AEROMEX.MX','WALMEX.MX','SAN.MX',
                'GMEXICOB.MX', 'Cetes']
    
    a_c = pd.DataFrame(columns = ['Asignación de Capital'], index=[tickers])
    a_c['Asignación de Capital'] = pesos
    
    return a_c



## Datos Historicos previos al 2015-12-31

In [89]:
start_date = '2011-12-31'
end_date = '2016-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [90]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port1 = pesos
#Port_prev
W_port1[:-1].sum()

1.5296

In [ ]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [58]:
inversion(w_1,Precio_i,Precio_f,100000)

Con una inversión de:  100000 Un apalancamiento de:  50666.23021316528 Tendrías una ganancia de : 38.79930610076623 % Con un saldo final de: 209123.6820640564


## Datos Historicos previos al 2016-12-31

In [73]:
start_date = '2011-12-31'
end_date = '2017-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [ ]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port2 = pesos
#Port_prev
W_port2[:-1].sum()

In [ ]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [ ]:
inversion(w_2,Precio_i,Precio_f,100000)

## Datos Historicos previos al 2017-12-31

In [ ]:
start_date = '2011-12-31'
end_date = '2018-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [ ]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port3 = pesos
#Port_prev
W_port3[:-1].sum()

In [ ]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [ ]:
inversion(w_3,Precio_i,Precio_f,100000)

## Datos Historicos previos al 2018-12-31

In [ ]:
start_date = '2011-12-31'
end_date = '2019-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [ ]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port4 = pesos
#Port_prev
W_port4[:-1].sum()

In [ ]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [ ]:
inversion(w_4,Precio_i,Precio_f,100000)

## Datos Historicos previos al 2019-12-31

In [ ]:
start_date = '2011-12-31'
end_date = '2020-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [ ]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port5 = pesos
#Port_prev
W_port5[:-1].sum()

In [ ]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [ ]:
inversion(w_5,Precio_i,Precio_f,100000)

## Datos Historicos previos al 2020-12-31

In [ ]:
start_date = '2011-12-31'
end_date = '2021-11-30'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [ ]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port6 = pesos
#Port_prev
W_port6[:-1].sum()

In [ ]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [ ]:
inversion(w_6,Precio_i,Precio_f,100000)

## Evaluacion 2016 

In [59]:
# Portafolio 2016
start_date = '2015-12-31'
end_date = '2016-12-31'
closes_16 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_16 = closes_16.pct_change().dropna()

In [60]:
a =closes_16[:1].sum()
a.sum()

8165.070388793945

In [61]:
w_1 =W_port1[:-1]
capital = 100000
Ret = ret_16
Precio_i = closes_16.iloc[1]
Precio_f = closes_16.iloc[-1]
Dest = np.floor(w_1*capital)
n_Acciones = Dest/Precio_i
n_Acciones

Symbols
XLK.MX           1.577392
VOO.MX           0.400789
VGT.MX           8.542195
VHT.MX           5.452311
VOX.MX          10.031424
BBVA.MX          0.000000
AC.MX            0.000000
AMXL.MX          0.000000
BIMBOA.MX        0.000000
GAPB.MX        701.921192
C.MX            66.386571
AEROMEX.MX     291.741238
WALMEX.MX        0.000000
SAN.MX          13.998816
GMEXICOB.MX      0.000000
Name: 2016-01-04 00:00:00, dtype: float64

In [62]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()

In [63]:
a

150627.12699890137

In [64]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [65]:
(Saldo_f/Saldo_i)-1

0.3880940722835349

In [66]:
closes_16.tail()

Symbols,XLK.MX,VOO.MX,VGT.MX,VHT.MX,VOX.MX,BBVA.MX,AC.MX,AMXL.MX,BIMBOA.MX,GAPB.MX,C.MX,AEROMEX.MX,WALMEX.MX,SAN.MX,GMEXICOB.MX
Date,,,,,,,,,,,,,,,
2016-12-26,1009.52002,NaN,NaN,NaN,NaN,138.163254,91.983780,11.295854,41.995281,130.713638,1288.974243,39.209999,32.701847,49.488338,43.485874
2016-12-27,1025.00000,4317.299805,123.760002,2657.899902,101.400002,138.163254,92.139702,11.330744,41.657742,130.596619,1263.265991,39.150002,32.640282,50.095276,43.602840
2016-12-28,1020.00000,4284.430176,122.589996,127.029999,100.610001,136.182434,93.352531,11.339467,42.351570,130.822876,1241.874512,39.250000,32.860165,48.802040,45.021965
2016-12-29,1004.00000,4220.000000,2534.840088,2633.270020,100.779999,136.281479,94.435410,11.452862,44.011131,132.328720,1224.052856,39.230000,33.027279,48.993462,44.998569
2016-12-30,1010.00000,4252.000000,2534.840088,126.650002,100.000000,137.668060,93.621071,11.365634,44.076763,133.015366,1225.654297,39.099998,32.587498,49.815151,44.000511


In [67]:
ret_16 = closes_16.pct_change().dropna()

## Evaluacion 2017

In [ ]:
# Portafolio 2017
start_date = '2016-12-31'
end_date = '2017-12-31'
closes_17 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_17 = closes_17.pct_change().dropna()

In [ ]:
a =closes_17[:1].sum()
a.sum()

In [ ]:
w_2 =W_port2[:-1]
capital = 100000
Ret = ret_17
Precio_i = closes_17.iloc[1]
Precio_f = closes_17.iloc[-1]
Dest = np.floor(w_2*capital)
n_Acciones = Dest/Precio_i
n_Acciones

In [ ]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()
a

In [ ]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [ ]:
(Saldo_f/Saldo_i)-1

In [ ]:
closes_17.tail()

## Evaluacion 2018

In [ ]:
# Portafolio 2018
start_date = '2017-12-31'
end_date = '2018-12-31'
closes_18 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_18 = closes_18.pct_change().dropna()

In [ ]:
a =closes_18[:1].sum()
a.sum()

In [ ]:
w_3 =W_port3[:-1]
capital = 100000
Ret = ret_18
Precio_i = closes_18.iloc[1]
Precio_f = closes_18.iloc[-1]
Dest = np.floor(w_3*capital)
n_Acciones = Dest/Precio_i
n_Acciones

In [ ]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()
a

In [ ]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [ ]:
(Saldo_f/Saldo_i)-1

In [ ]:
closes_18.tail()

## Evaluacion 2019

In [ ]:
# Portafolio 2019
start_date = '2018-12-31'
end_date = '2019-12-31'
closes_19 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_19 = closes_19.pct_change().dropna()

In [ ]:
a =closes_19[:1].sum()
a.sum()

In [ ]:
w_4 =W_port4[:-1]
capital = 100000
Ret = ret_19
Precio_i = closes_19.iloc[1]
Precio_f = closes_19.iloc[-1]
Dest = np.floor(w_4*capital)
n_Acciones = Dest/Precio_i
n_Acciones

In [ ]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()
a

In [ ]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [ ]:
(Saldo_f/Saldo_i)-1

In [ ]:
closes_19.tail()

## Evaluacion 2020

In [ ]:
# Portafolio 2020
start_date = '2019-12-31'
end_date = '2020-12-31'
closes_20 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_20 = closes_20.pct_change().dropna()

In [ ]:
a =closes_20[:1].sum()
a.sum()

In [ ]:
w_5 =W_port5[:-1]
capital = 100000
Ret = ret_20
Precio_i = closes_20.iloc[1]
Precio_f = closes_20.iloc[-1]
Dest = np.floor(w_5*capital)
n_Acciones = Dest/Precio_i
n_Acciones

In [ ]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()
a

In [ ]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [ ]:
(Saldo_f/Saldo_i)-1

In [ ]:
closes_20.tail()

## Evaluacion 2021

In [ ]:
# Portafolio 2021
start_date = '2020-12-31'
end_date = '2021-11-30'
closes_21 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_21 = closes_21.pct_change().dropna()

In [ ]:
a =closes_21[:1].sum()
a.sum()

In [ ]:
w_6 =W_port6[:-1]
capital = 100000
Ret = ret_21
Precio_i = closes_21.iloc[1]
Precio_f = closes_21.iloc[-1]
Dest = np.floor(w_6*capital)
n_Acciones = Dest/Precio_i
n_Acciones

In [ ]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()
a

In [ ]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [ ]:
(Saldo_f/Saldo_i)-1

In [ ]:
closes_21.tail()